In [ ]:
graph_folder:str = 'data'
edition_file: str = 'temp/yeast_edit'
matrix:str = 'temp/matrix.json'
mash_matrix:str = 'temp/mash.json'

In [ ]:
# Colorblind-friendly palette
colors = {
    'blue':    '#377eb8', 
    'orange':  '#ff7f00',
    'green':   '#4daf4a',
    'pink':    '#f781bf',
    'brown':   '#a65628',
    'purple':  '#984ea3',
    'gray':    '#999999',
    'red':     '#e41a1c',
    'yellow':  '#dede00'
} 

In [ ]:
from seaborn import heatmap, boxplot,stripplot, clustermap, scatterplot, regplot, color_palette
from numpy import triu,array,add,zeros,column_stack, nan
from matplotlib import pyplot as plt
from os import listdir, path
from collections import Counter
from matplotlib.patches import Rectangle
from matplotlib import patches as mpatches
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from subprocess import run, PIPE
from scipy.spatial.distance import euclidean
from statistics import mean,median
from pandas import DataFrame,wide_to_long
from pgGraphs import Graph
from math import floor
from os import system, remove, path
from json import load, dump
from tharospytools.matplotlib_tools import get_palette_from_list

Manipulating graph names

In [ ]:
all_graphs: list = sorted(listdir(graph_folder))
all_graphs.insert(0, all_graphs.pop())
print(all_graphs)

In [ ]:
gfa_graph:Graph = Graph(
    gfa_file=f'data/{all_graphs[0]}'
)

Computing pairwise mash distance

In [ ]:
graph_paths:list[str] = sorted(list(gfa_graph.paths.keys()))

In [ ]:
if path.exists(mash_matrix):
    mash: list[list[int]] = load(open(mash_matrix,'r',encoding='utf-8'))
else:
    mash:list[list[float]] = [
            [
            0 for _ in range(len(graph_paths))
        ] for _ in range(len(graph_paths))
    ]

    for i,path_name_A in enumerate(graph_paths):
        for j,path_name_B in enumerate(graph_paths):
            mash[i][j]=float(run(['mash', 'dist',f'temp/{path_name_A}.fa',f'temp/{path_name_B}.fa'], stdout=PIPE).stdout.decode('utf-8').split()[2])
    dump(mash,open(mash_matrix,'w',encoding='utf-8'))

In [ ]:
for row in mash:
    print("\t".join([str(elt) for elt in row]))

Computing mean and median of mash distance to every other for each sequence

In [ ]:
mean_mash:dict = {
    path_name:mean(mash[i]) for i,path_name in enumerate(graph_paths)
}

median_mash:dict = {
    path_name:median(mash[i]) for i,path_name in enumerate(graph_paths)
}

sum_mash:dict = {
    path_name:sum(mash[i]) for i,path_name in enumerate(graph_paths)
}

Computing distances

In [ ]:
# First, we compute edition to be sure to have a list of breakpoints
distance_martix:list[list[int]] = [[0 for _ in all_graphs] for _ in all_graphs]

if path.exists(matrix):
    distance_martix: list[list[int]] = load(open(matrix,'r',encoding='utf-8'))
else:
    for i,graph_A in enumerate(all_graphs):
        for j,graph_B in enumerate(all_graphs):
            if i < j:
                # Compute edition
                system(f'pancat edit data/{graph_A} data/{graph_B} -o {edition_file}_{i}_{j}.json')
                # Load edit file in memory
                editions:dict[str,list] = load(open(f"{edition_file}_{i}_{j}.json",'r',encoding='utf-8'))
                # Count editions
                edit_count:int = sum([sum([len(edit_list) for edit_list in edition_classes.values()]) for edition_classes in editions.values()])
                # add count to the two positions
                distance_martix[i][j] = edit_count
                distance_martix[j][i] = edit_count
                # delete temp file
                #remove(edition_file)
                
    dump(distance_martix,open(matrix,'w',encoding='utf-8'))

In [ ]:
for row in distance_martix:
    print("\t".join([str(elt) for elt in row]))

In [ ]:
labels:list[str] = ['PGGB' if 'pggb' in label else label.split('_')[3] for label in all_graphs]
green_pal = color_palette("dark:#5A9", as_cmap=True)
colors = ['white'] + get_palette_from_list([sum_mash[x] for x in graph_paths],cmap_name='Greens')
fig = clustermap(distance_martix, linewidth=0.5, annot=False, xticklabels=['PGGB']+[round(sum_mash[x],3) for x in graph_paths], yticklabels=labels,cmap=green_pal,col_colors=colors,row_colors=colors)
fig.tick_params(length=0)
fig.ax_heatmap.set_yticklabels(fig.ax_heatmap.get_yticklabels(), rotation=0)
plt.show()

In [ ]:
categories: str = ['String', 'SuperString', 'SubString', 'SubSuffix', 'SuperPrefix',
                    'SubPrefix', 'SuperSuffix', 'OverlapSuffixPrefix', 'OverlapPrefixSuffix','Splits','Merges']




# fig, axs = plt.subplots(ncols=len(categories))
size_square:int = len(all_graphs)
select:list|None = None #["seq3"]

table: dict[str, list[list[int]]] = {category: [
    [0 for _ in range(len(all_graphs))] for _ in range(len(all_graphs))] for category in categories}

dipaths:list = list()
deltas:list = list()
for i, graph_1 in enumerate(all_graphs):
    print(f"Starting comparisons on graph {graph_1} ({i+1}/{size_square})")
    for j, graph_2 in enumerate(all_graphs):
        results, all_dipaths,deltas_counter = perform_edition(
            files=[path.join(graph_folder, graph_1), path.join(graph_folder, graph_2)],
            gfa_versions=['GFA1.1', 'GFA1.1'],
            selection=select,
            distance_mode='graph_level'
            )
        dipaths.append(all_dipaths)
        deltas.append(deltas_counter)
        for category in categories:
            table[category][i][j] = results[category]

In [ ]:
graph_names:list = [x.split('.')[0] for x in all_graphs]

for i, (category, datas) in enumerate(table.items()):
    # axs[i].set_title(category)
    fig = plt.figure(figsize=(size_square+2, size_square+2))
    plt.title(category)
    ax = heatmap(datas, linewidth=0.5, annot=True,
                        cbar=False, xticklabels=graph_names, yticklabels=graph_names)  # ax=axs[i]
    
    wanted_index = graph_names.index(final_name)
    x, y, w, h = 0, wanted_index, size_square, 1
    for _ in range(2):
        ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='crimson', lw=4, clip_on=False))
        x, y = y, x # exchange the roles of x and y
        w, h = h, w # exchange the roles of w and h
    ax.tick_params(length=0)
    plt.savefig(path.join(output_folder,f"graph_{category}.png"), bbox_inches='tight')
    save_dynamic_fig(fig,path.join(output_folder,f"graph_{category}.pkl"))
    plt.clf()

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

plt.rcParams['grid.color'] = (0.5, 0.5, 0.5, 0.3)

for category,data in table.items():
    # setting distance_threshold=0 ensures we compute the full tree.
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
    model = model.fit(data)
    plt.title(f"Hierarchical Clustering Dendrogram on {category}")
    plt.tick_params(length=0)
    # plot the top three levels of the dendrogram
    plot_dendrogram(model, truncate_mode="level", p=3, labels=graph_names,leaf_rotation=90.,leaf_font_size=8.)
    plt.box(False)
    plt.ylabel('Cluster distance')
    plt.yticks(fontsize=8)
    plt.grid(axis = 'y',linestyle='--')
    plt.show()
    save_dynamic_fig(fig,path.join(output_folder,f"graph_editions_{category}.pkl"))
    plt.savefig(path.join(output_folder,f"graph_editions_{category}.png"), bbox_inches='tight')
    plt.clf()


In [ ]:
# Computing euclidian distance between edition patterns

matrix = add(array(table["Merges"]),array(table["Splits"]))

mx1 = [[elt for x,elt in enumerate(row) if x!=y] for y,row in enumerate(matrix)]
mx2 = [[elt if x!=y and 'pggb' not in graph_names[x] else None for x,elt in enumerate(row)] for y,row in enumerate(matrix)]
mx3 = [[elt if x!=y and 'pggb' in graph_names[x] else None for x,elt in enumerate(row)] for y,row in enumerate(matrix)]

# For each line in the matrix, compute the distance with every other, and keep the one with the lowest mean distance
lowest_distance:list = [mean([euclidean(reference,line) for i,line in enumerate(matrix) if i !=j]) for j,reference in enumerate(matrix)]
idx:int = lowest_distance.index(min(lowest_distance))

df1 = DataFrame(mx1).T
df1 = df1.rename(columns={k: f'Data{k+1}' for k in range(len(mx1))}).reset_index()
df1 = wide_to_long(df1, stubnames = ['Data'], i = 'index', j = 'ID').reset_index()[['ID', 'Data']]

df2 = DataFrame(mx2).T
df2 = df2.rename(columns={k: f'Data{k+1}' for k in range(len(mx2))}).reset_index()
df2 = wide_to_long(df2, stubnames = ['Data'], i = 'index', j = 'ID').reset_index()[['ID', 'Data']]

daf = DataFrame(mx3).T
daf = daf.rename(columns={k: f'Data{k+1}' for k in range(len(mx3))}).reset_index()
daf = wide_to_long(daf, stubnames = ['Data'], i = 'index', j = 'ID').reset_index()[['ID', 'Data']]

fig, ax = plt.subplots()
fig.set_size_inches(len(graph_names)+2,8)
pal = {str(i+1):'red' if 'pggb' in graph_names[i] else 'orange' for i in range(len(graph_names))}
boxplot(x='ID', y = 'Data', data = df1, boxprops={'alpha': 0.4},palette=pal)
stripplot(data=df2, x="ID", y="Data", dodge=True, ax=ax, color=colors['orange'])
stripplot(data=daf, x="ID", y="Data", dodge=True, ax=ax, color=colors['red'])
plt.xlabel("Graphs")
ax.set_xticklabels(graph_names)
for lab in ax.get_xticklabels():
   if lab.get_text() == graph_names[idx]:
      lab.set_fontweight('bold')
red_patch = mpatches.Patch(color=colors['red'], label='PGGB')
orange_patch = mpatches.Patch(color=colors['orange'], label='MGC')
plt.legend(handles=[red_patch, orange_patch])
plt.ylabel("Edit distance")
save_dynamic_fig(fig,path.join(output_folder,f"graph_euclidian.pkl"))
plt.savefig(path.join(output_folder,f"graph_euclidian.png"), bbox_inches='tight')
plt.show()

In [ ]:
# Computing editions
mask = triu(matrix)

plt.figure(figsize=(size_square+2, size_square+2))
plt.title("Editions")
ax = heatmap(matrix, linewidth=0.5, annot=True,
                    cbar=False, mask=mask, xticklabels=graph_names, yticklabels=graph_names)  # ax=axs[i]
wanted_index = graph_names.index(final_name)
x, y, w, h = 0, wanted_index, size_square, 1
x, y = y, x # exchange the roles of x and y
w, h = h, w # exchange the roles of w and h
ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='crimson', lw=4, clip_on=False))

ax.tick_params(length=0)
plt.savefig(path.join(output_folder,f"graph_editions_triangle.png"), bbox_inches='tight')
plt.show()
plt.clf()

Clustermap des éditions

In [ ]:
fig = clustermap(matrix, linewidth=0.5, annot=True, xticklabels=graph_names, yticklabels=graph_names,metric="correlation")
fig.tick_params(length=0)
fig.savefig(path.join(output_folder,f"graph_clustering.png"), bbox_inches='tight')
fig.ax_heatmap.set_yticklabels(fig.ax_heatmap.get_yticklabels(), rotation=0)
plt.clf()

In [ ]:
# Displaying events sizes        
from numpy import cumsum

for i,dc in enumerate(deltas):
    fig = plt.figure(figsize=(20,5))
    plt.title("Lengths of $\Delta_{events}$")
    plt.yscale("log")
    plt.xscale("log")
    for category,compteur in dc.items():
        try:
            x=list(range(max(list(compteur.keys()))))
            y=cumsum([compteur[i] if i in compteur else 0 for i in range(max(list(compteur.keys())))])
            plt.plot(x,y, label = category, alpha=0.6)
        except:
            print(f"{category} -> {compteur}")

    plt.ylabel("Cumulative sum of $\Delta_{events}$")
    plt.legend(loc = 'upper left')
    plt.savefig(path.join(output_folder,f"deltas_lengths_{i}.png"), bbox_inches='tight')
    plt.clf()

Analyse de la corrélation entre la moyenne des mash distance à la référence et la proximité du graphe *minigraph-cactus* avec la plus basse moyenne au graphe *PGGB*.

In [ ]:
"""
mash:dict = {}

for file in listdir(pipeline_folder):
    with open(path.join(pipeline_folder,file),'r',encoding='utf-8') as pipl:
        all_lines:list = pipl.readlines()
        reference,alternates = all_lines[0].split()[1],[line.split()[1] for line in all_lines[1:]]
        mash[file.split('.')[0]]=[float(run(['mash', 'dist',reference,alt], stdout=PIPE).stdout.decode('utf-8').split()[2]) for alt in alternates]

mean_mash:dict = {k:mean(v) for k,v in mash.items()}
median_mash:dict = {k:median(v) for k,v in mash.items()}
"""

In [ ]:
"""
dtf = DataFrame([mean_mash]).T
dtf = dtf.rename(columns={0:'mash_dist'})
dtf["pggb_dist"] = [matrix[graph_names.index(x)][graph_names.index(final_name)] for x,_ in dtf.iterrows()]

dtf2 = DataFrame([median_mash]).T
dtf2 = dtf2.rename(columns={0:'mash_dist'})
dtf2["pggb_dist"] = [matrix[graph_names.index(x)][graph_names.index(final_name)] for x,_ in dtf2.iterrows()]

#m, b = np.polyfit(dtf['mash_dist'], dtf['pggb_dist'], 1)
#plt.plot(dtf['mash_dist'], m*dtf['mash_dist']+b, color='red')
ax1 = fig.add_subplot()
regplot(x=dtf['mash_dist'], y=dtf['pggb_dist'],scatter_kws={'alpha':0.3},line_kws=dict(alpha=0.3, color=colors['orange'], linewidth=3), ax=ax1)
scatterplot(data=dtf, x="mash_dist", y="pggb_dist", color=colors['red'], ax=ax1)
#plt.xlabel("Mean mash distance")
ax1.set_ylabel("Distance to PGGB graph")
ax2 = ax1.twinx()
regplot(x=dtf2['mash_dist'], y=dtf2['pggb_dist'],scatter_kws={'alpha':0.3},line_kws=dict(alpha=0.3, color=colors['purple'], linewidth=3), ax=ax2)
scatterplot(data=dtf2, x="mash_dist", y="pggb_dist", color=colors['blue'], ax=ax2)
ax2.set_ylabel("Distance to PGGB graph")
plt.savefig(path.join(output_folder,f"mash_correlation.png"), bbox_inches='tight')
plt.clf()
"""

In [ ]:
for i,seq_dipaths in enumerate(dipaths):
    # Plotting repartition of edits along the reference
    categories_left: str = ['Overlaps', 'Equivalences', 'Inclusions']
    categories_right: str = [('Splits',colors['red']),('Merges',colors['green'])]

    sampling:int = 100
    lengths:int = [int(p.datas['stop_offset'])-int(p.datas['start_offset']) for p in Graph(gfa_file=path.join(graph_folder, all_graphs[1]),gfa_type='GFA1.1',with_sequence=True).get_path_list()]
    counts = [{x:0 for x in categories_left+['Splits']+['Merges']} for _ in range(sampling)]
    # here, iterating on graphs
    if i//len(graph_names) != i%len(graph_names):
        # we don't compute if we compare the same graphs
        for j,dipath in enumerate(seq_dipaths):
            # we're iterating on the dipaths of a single comparison at this level
            curr_length = lengths[j]
            # compute sectors here
            for edition_list in dipath.edition.values():
                curr_index = floor((edition_list[0].offsets_ref[0]/curr_length)*sampling)
                if curr_index > sampling-1:
                    print(f"WARNING! Index value = {curr_index}")
                    curr_index = sampling-1
                counts[curr_index]['Merges'] += max(0, len(edition_list)-1)
                for edit_element in edition_list:
                    if type(edit_element).__name__ in ['OverlapPrefixSuffix','OverlapSuffixPrefix']:
                        counts[curr_index]['Overlaps'] += 1
                        if type(edit_element).__name__ == 'OverlapPrefixSuffix':
                            counts[curr_index]['Splits'] +=1
                    elif type(edit_element).__name__ in ['SubSuffix','SubPrefix','SuperString','SubString','SuperPrefix','SuperSuffix']:
                        counts[curr_index]['Inclusions'] += 1
                        if type(edit_element).__name__ in ['SuperPrefix','SuperString']:
                            counts[curr_index]['Splits'] +=1
                    elif type(edit_element).__name__ == 'String':
                        counts[curr_index]['Equivalences'] += 1
            
        fig = plt.figure(figsize=(20,5))
        fig.suptitle(f"Distribution of events for {graph_names[i//len(graph_names)]} against {graph_names[i%len(graph_names)]}")

        x = list(range(sampling))

        ax1 = fig.add_subplot()
        ax1.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)
        
        for category in categories_left:
            ax1.plot(x,[counts[sector][category] for sector in x], label = category, alpha=0.6)
        
        ax2 = ax1.twinx()
        ax2.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)

        for category,color in categories_right:
            ax2.plot(x,[counts[sector][category] for sector in x], color=color, label = category,linestyle='dashed', alpha=0.8)

        ax1.set_ylabel("Events")
        ax2.set_ylabel("Editions")
        ax1.legend(loc = 'upper left')
        ax2.legend(loc = 'upper right')
        ax1.set_xlim(0,99)
        ax2.set_xlim(0,99)
        
        save_dynamic_fig(ax1,path.join(output_folder,f"graph_{graph_names[i//len(graph_names)]}_{graph_names[i%len(graph_names)]}_ax1.pkl"))
        save_dynamic_fig(ax2,path.join(output_folder,f"graph_{graph_names[i//len(graph_names)]}_{graph_names[i%len(graph_names)]}_ax2.pkl"))
        save_dynamic_fig(fig,path.join(output_folder,f"graph_{graph_names[i//len(graph_names)]}_{graph_names[i%len(graph_names)]}_fig.pkl"))
        
        plt.savefig(path.join(output_folder,f"graph_{graph_names[i//len(graph_names)]}_{graph_names[i%len(graph_names)]}.png"), bbox_inches='tight')
        plt.show()
        plt.clf()
